In [1]:
import tensorflow as tf
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Model

import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans

In [2]:
path = 'Traffic Sign Recognition'
lab = pd.read_csv(path + '/labels.csv')

In [3]:
d = dict()
class_labels = dict()
for dirs in os.listdir(path + '/myData'):
    count = len(os.listdir(path+'/myData/'+dirs))
    d[dirs+' => '+lab[lab.ClassId == int(dirs)].values[0][1]] = count
    class_labels[int(dirs)] = lab[lab.ClassId == int(dirs)].values[0][1]

In [4]:
# input image dimensions
img_rows, img_cols = 32, 32
# The images are RGB.
img_channels = 3
nb_classes = len(class_labels.keys())

datagen = ImageDataGenerator()
data = datagen.flow_from_directory(path + '/myData',
                                    target_size=(32, 32),
                                    batch_size=73139,
                                    class_mode='categorical',
                                    shuffle=True )

Found 73139 images belonging to 43 classes.


In [5]:
X , y = data.next()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)

In [6]:
resnet = ResNet50(weights= None, include_top=False, input_shape= (img_rows,img_cols,img_channels))
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation= 'softmax')(x)
model = Model(inputs = resnet.input, outputs = predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
n_epochs = 50
history =  model.fit(X_train, y_train,  batch_size = 32, epochs = n_epochs, verbose = 1, 
              validation_data = (X_test, y_test))

Epoch 1/50
1829/1829 [==============================] - 147s 74ms/step - loss: 4.3179 - accuracy: 0.1067 - val_loss: 58.1082 - val_accuracy: 0.0723
Epoch 2/50
1829/1829 [==============================] - 137s 75ms/step - loss: 3.7285 - accuracy: 0.1254 - val_loss: 3.4078 - val_accuracy: 0.1098
Epoch 3/50
1829/1829 [==============================] - 137s 75ms/step - loss: 3.4257 - accuracy: 0.1517 - val_loss: 10.2677 - val_accuracy: 0.1399
Epoch 4/50
1829/1829 [==============================] - 137s 75ms/step - loss: 2.8864 - accuracy: 0.2381 - val_loss: 11.6311 - val_accuracy: 0.2130
Epoch 5/50
1829/1829 [==============================] - 137s 75ms/step - loss: 1.9653 - accuracy: 0.4321 - val_loss: 5.2521 - val_accuracy: 0.4629
Epoch 6/50
1829/1829 [==============================] - 134s 73ms/step - loss: 1.3362 - accuracy: 0.6035 - val_loss: 0.9218 - val_accuracy: 0.7023
Epoch 7/50
1829/1829 [==============================] - 129s 71ms/step - loss: 0.5839 - accuracy: 0.8186 - val_loss

In [8]:
model.save('traffic_sign_classifier.h5')

C:\Users\colin\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
